In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

from scipy.sparse.linalg import svds
from numpy.linalg import svd 
from scipy.sparse import csr_matrix
import time

## submit 0

### base matrix

In [131]:
train = pd.read_csv('/data/agoryach/datagym-recsys-01/big-hw-02/input/train_data_full.csv')

In [132]:
train.user_id.nunique(), train.primary_video_id.nunique()

(407381, 5266)

In [133]:
aggr1 = {
    'session_duration': 'max', 
    'video_duration': 'max'
}

train_s = train.groupby(['user_id', 'primary_video_id'])\
               .agg(aggr1).reset_index().reset_index()

train_s['session_duration_clean'] = train_s[['session_duration', 'video_duration']].min(axis=1)
train_s['watching_percentage'] = train_s['session_duration_clean']/train_s['video_duration']
train_s = train_s[['user_id', 'primary_video_id', 'watching_percentage']]

In [401]:
%%time
utilMat_r, users_index, items_index = create_utility_matrix(train_s)

CPU times: user 7min 24s, sys: 45.8 s, total: 8min 10s
Wall time: 8min 10s


In [403]:
by_user = train_s[['user_id', 'primary_video_id']].groupby('user_id').count().reset_index()
freq_users = by_user[by_user['primary_video_id']>3].user_id.unique()

by_item = train_s[['user_id', 'primary_video_id']].groupby('primary_video_id').count().reset_index()
freq_items = by_item[by_item['user_id'] > 10].primary_video_id.unique()

train_ss = train_s[(train_s['user_id'].isin(freq_users))&
                   (train_s['primary_video_id'].isin(freq_items))]

In [402]:
sparse_m = csr_matrix(utilMat_r)

MemoryError: Unable to allocate 32.0 GiB for an array with shape (2145268346, 2) and data type int64

In [380]:
def create_utility_matrix(data, formatizer={'user': 0, 'item': 1, 'value': 2}):
    '''
        функция создания матрицы
    '''
    itemField = formatizer['item'] 
    userField = formatizer['user']
    valueField = formatizer['value']
    userList = data.iloc[:, userField].tolist()
    itemList = data.iloc[:, itemField].tolist()
    valueList = data.iloc[:, valueField].tolist()
    users = list(set(data.iloc[:, userField]))
    items = list(set(data.iloc[:, itemField]))
    users_index = {users[i]: i for i in range(len(users))}
    pd_dict = {item: [np.nan for i in range(len(users))] for item in items}
    for i in range(0, len(data)):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]
        pd_dict[item][users_index[user]] = value
    X = pd.DataFrame(pd_dict)
    X.index = users
    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    return X, users_index, items_index

CPU times: user 3min 42s, sys: 27.9 s, total: 4min 10s
Wall time: 4min 10s


In [399]:
U, s, V = svds(sparse_m, k=10)

In [400]:
s = np.diag(s)
s

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [382]:
def svd_fillna_shift_norm(train, k):
    '''
        svd с нормировкой + заполнениями и сдвигом
    '''
    
    print(f'start {time.ctime()}')
    utilMat = train.copy()
#     # маска из нанов
#     mask = np.isnan(utilMat)
#     masked_arr = np.ma.masked_array(utilMat, mask)
#     item_means = np.mean(masked_arr, axis=0)
#     print(f'done mask {time.ctime()}')
#     # замена средним для нанов
#     utilMat = masked_arr.filled(item_means)
#     x = np.tile(item_means, (utilMat.shape[0], 1))
#     print(f'done fillna {time.ctime()}')
#     # вычитание
#     utilMat = utilMat - x
#     total_mean = np.mean(utilMat)
#     col_means = np.mean(utilMat, axis=0) - total_mean
#     row_means = np.mean(utilMat, axis=1) - total_mean
#     utilMat = utilMat - row_means[:, np.newaxis] - col_means - total_mean
#     print(f'done centering {time.ctime()}')
#     row_norms = np.sqrt(np.sum(utilMat*utilMat, axis=1))[:, np.newaxis]
#     col_norms = np.sqrt(np.sum(utilMat*utilMat, axis=0))[np.newaxis, :]
#     utilMat = utilMat / np.sqrt(row_norms) / np.sqrt(col_norms)
#     print(f'done norm {time.ctime()}')
    U, s, V = np.linalg.svd(utilMat, full_matrices=False)
    print(f'done svd {time.ctime()}')
    s = np.diag(s)
    s = s[0:k, 0:k]
    U = U[:, 0:k]
    V = V[0:k, :]
    s_root = np.sqrt(s)
    Usk = np.dot(U, s_root)
    skV = np.dot(s_root, V)
    UsV = np.dot(Usk, skV)
    print(f'done dot {time.ctime()}')
#     # восстановление
#     UsV = UsV * np.sqrt(row_norms) * np.sqrt(col_norms)
#     UsV = UsV + row_means[:, np.newaxis] + col_means + total_mean
#     UsV = UsV + x
    print(f'done recover {time.ctime()}')
    return UsV

In [383]:
%%time
#svdout = svd_fillna_shift_norm(utilMat, k=10)
UsV = svd_fillna_shift_norm(utilMat_r.copy(), k=20)

start Thu Jun  4 21:18:59 2020


ValueError: On entry to DLASCL parameter number 4 had an illegal value

In [24]:
#UsV.dump("/data/agoryach/datagym-recsys-01/big-hw-02/input/UsV_baseline.dat")

### cold start

In [384]:
test = pd.read_csv('/data/agoryach/datagym-recsys-01/big-hw-02/input/sample_submission_full.csv')

In [385]:
cold_users = [x for x in test['user_id'].unique() if x not in users_index.keys()]

In [386]:
test['user_id'].nunique(), len(cold_users)

(187183, 52714)

In [387]:
# заполнить 0 те айтемы, с котооорыми юзер уже взаимодействовал
UsV_temp = UsV.copy()
m = np.isnan(utilMat_r)
masked_arr = np.ma.masked_array(utilMat_r, ~m)
UsV_temp.fill_value = 0
UsV_temp.mask = masked_arr.mask
UsV_temp = UsV_temp.filled()

In [388]:
# get top 10 for hot users
top10 = np.argpartition(-UsV_temp, 10, axis=1)[:, :10]

In [389]:
rev_items_index = {v:k for k,v in items_index.items()}
rev_user_index = {v:k for k,v in users_index.items()}

In [390]:
result = pd.DataFrame([], index=users_index.keys()).reset_index().rename(columns={'index':'user_id'})
result['item_id'] = pd.Series(list(top10))
result['primary_video_id'] = result['item_id'].apply(lambda x: ' '.join([str(rev_items_index[i]) for i in x]))
result.drop('item_id', axis=1, inplace=True)

KeyError: 4528

In [ ]:
item_info = pd.read_csv('/data/agoryach/datagym-recsys-01/big-hw-02/input/video_meta_data_full.csv')

In [ ]:
# самые популярные товары
item_pop_matrix = pd.DataFrame([])
item_pop_matrix['item_id'] = items_index.keys()
item_pop_matrix['cnt_not_null'] = (utilMat_r.T > 0).sum(axis=1).reset_index(drop=True)

# среднее по товарам
mask = np.isnan(utilMat_r)
masked_arr = np.ma.masked_array(utilMat_r, mask)
item_means = np.mean(masked_arr, axis=0)

item_pop_matrix['item_means'] = item_means

In [ ]:
n_item_count = 8000

top10_items = item_pop_matrix[item_pop_matrix['cnt_not_null'] > n_item_count]\
.sort_values('item_means', ascending=False)\
.head(10)['item_id'].values

len(top10_items), item_info[item_info['primary_video_id'].isin(top10_items)].title.unique()

In [ ]:
result2 = pd.DataFrame(cold_users, columns=['user_id'])
result2['primary_video_id'] = ' '.join([str(x) for x in top10_items])

In [ ]:
finalDF = result.append(result2).reset_index(drop=True)
finalDF = finalDF[finalDF['user_id'].isin(test['user_id'].unique())].reset_index(drop=True)

In [ ]:
finalDF.to_csv('/data/agoryach/datagym-recsys-01/big-hw-02/output/submission_2.csv', index=False)

In [ ]:
# сделать функцию map k
# cross val params:
    # - svd k
    # mean by user/item
    # n_item_count